In [1]:
# from read_viva import *
# # import matplotlib.image as mpimg
# # import matplotlib.pyplot as plt

In [2]:
# img_list, box_list = load_viva('/home/cyrano5614/Downloads/detectiondata/train/')

In [3]:
# generate = generate_batch(img_list, box_list, (128, 128), 10, negative=True)


In [4]:
# for i in range(1):
#     batch_imgs, batch_labels = next(generate)
#     batch_visualize(batch_imgs, batch_labels)

In [5]:
from viva.cnn.networks.lenet import LeNet
from viva.cnn.networks.xception_transfer import Xception_Transfer
from viva.bottleneck.extract_bottleneck_features import extract_bottleneck_Xception, extract_features_labels
from read_viva import load_viva, generate_batch
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import cv2
from pipeline import pretrained_model

img_size = (100, 100)
batch_size = 10
path = '/home/cyrano5614/Downloads/detectiondata/train/'
img_list, box_list = load_viva(path)
# Splitting into 60/20/20
train_imgs, test_imgs, train_boxes, test_boxes = train_test_split(
    img_list, box_list, test_size=0.2)
train_imgs, valid_imgs, train_boxes, valid_boxes = train_test_split(
    train_imgs, train_boxes, test_size=0.25)

# train_gen = 
bottleneck_model = pretrained_model('Xception', img_size)

train_generator = generate_batch(train_imgs, train_boxes, img_size=img_size, bottleneck=True, model=bottleneck_model, batch_size=batch_size)
result = next(train_generator)
result[0].shape
# print(type(train_generator))
# bottleneck_train_convert = extract_features_labels(bottleneck_model, next(train_generator), batch_size=batch_size)
# valid_generator = generate_batch(
#     valid_imgs, valid_boxes, img_size=img_size, batch_size=batch_size)
# test_generator = generate_batch(
#     test_imgs, test_boxes, img_size=img_size, batch_size=batch_size)

# model_name = 'xception'
# bottleneck_features = extract_bottleneck_Xception(train_generator, valid_generator, test_generator,
#                                                   img_shape=(img_size[0], img_size[1], 3), batch_size=batch_size)
# model = Xception_Transfer.build(bottleneck_features['train']['features'],
#                                 bottleneck_features['train']['labels'])
# bottleneck_features['train']['features']
# bottleneck_train_generator = bottleneck_features
# bottleneck_valid_generator = extract_features_labels(model, valid_generator, batch_size=batch_size)
# bottleneck_test_generator = extract_features_labels(model, test_generator, batch_size=batch_size)



Using TensorFlow backend.


(10, 3, 3, 2048)

In [25]:
x, y, z = bottleneck_model.output.shape[1:]
int(x)


3

In [27]:
len(next(train_generator))

2

In [17]:
len(np.unique(result[1]))

2